In [ ]:
api_key = ""

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
##intializing the Pinecone client
pc = Pinecone(api_key = api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        #we will use the sentence trasformer as embedding model, so it default convert into 384
        dimension = 384,    #dimension of dense vector
        metric = 'dotproduct', ##sparse values supported only for dotproducts
        spec = ServerlessSpec(cloud = 'aws', region = "us-east-1"),
    )

In [7]:
index = pc.Index(index_name)
index

c:\Jasweer\my_project\Langchain\Lvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [12]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited NewYork",
    "In 2021, I visited New Orleans",
]
# performing tfidf on the senteses
bm25_encoder.fit(sentences)
## store the values to a json file
bm25_encoder.dump("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 45.98it/s]


In [13]:
retriever = PineconeHybridSearchRetriever(embeddings= embeddings, sparse_encoder=bm25_encoder, index = index)

In [14]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001F1C1CE5280>, index=<pinecone.db_data.index.Index object at 0x000001F1FCECA5D0>)

In [15]:
retriever.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited NewYork",
        "In 2021, I visited New Orleans",
    ]
)


100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


In [17]:
retriever.invoke("In which year I visit the NewYork?")

[Document(metadata={'score': 0.533768058}, page_content='In 2022, I visited NewYork'),
 Document(metadata={'score': 0.227168605}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.169513285}, page_content='In 2023, I visited Paris')]